In [ ]:
# | default_exp cli

# Command Line Interface

In [ ]:
# | hide
from fastcore.test import *
import jupyter_black
import tempfile

In [ ]:
# | hide
jupyter_black.load()

In [ ]:
# | export
import click
from pathlib import Path
from sal.core import Data
from sal.loaders import xml_file_to_data
from sal.codegen import (
    Sal,
    FrontMatterInMemoryTemplateLoader,
    Renderer,
    JinjaTemplateRenderer,
    MissingTemplate,
)

## Generating code


Up until now, we've been developing the code we need to generate code. Now it's time to wrap that code in a easy to use function to use as a command line interface. This cli will mirror the args of this function so:

- it accepts an xml file path as input
- it accepts a template directory as an input

Also, when a certain template does not exist, it will create one with a default template `Renderer.DEFAULT_TEMPLATE`.

In [ ]:
# | export
def render(file, templates):
    try:
        repository = FrontMatterInMemoryTemplateLoader.from_directory(templates)
        renderer = Renderer(repository=repository, renderer=JinjaTemplateRenderer())
        sal = Sal(renderer)

        struct: Data = xml_file_to_data(file)
        return sal.process(struct)
    except MissingTemplate as e:
        path = Path(templates) / f"{e.name}.jinja2"
        path.write_text(Renderer.DEFAULT_TEMPLATE)
        return render(file, templates)

In [ ]:
# | hide
model = """
---
reference:  "sigla-{{ node.attrs.name | lower }}-model"
---
class {{ name }}Model(models.Model): # {{ reference }}
    {% for child in children -%}
    {{ child | render }}
    {% endfor %}
"""

field = """
---
reference:  "sigla-{{ node.name | lower }}-model"
---
{{ name }} = models.{{ type | title }}Field() 
"""


destination = tempfile.NamedTemporaryFile()

xml = f"""
<to-file to="{destination.name}">
    <model name="User">
        <field name="id" type="integer"/>
        <field name="username" type="char"/>
        <field name="email" type="email"/>
    </model>
</to-file>"""

In [ ]:
# | hide
# prepare files and directories

Path(destination.name).unlink(missing_ok=True)

# create template dir
path = Path("/tmp/templates")
path.mkdir(exist_ok=True)

# create xml file
Path("/tmp/sal.xml").write_text(xml)

# create template files
Path("/tmp/templates/model.jinja2").write_text(model)
Path("/tmp/templates/field.jinja2").write_text(field)

106

In [ ]:
# render
render("/tmp/sal.xml", "/tmp/templates")


result = Path(destination.name).read_text().strip()

test_eq(
    result,
    """
class UserModel(models.Model): # sigla-user-model
    id = models.IntegerField()
    username = models.CharField()
    email = models.EmailField()
""".strip(),
)

---

In [ ]:
# | export
@click.command()
@click.option("--count", default=1, help="Number of greetings.")
@click.option("--name", prompt="Your name")
def hello(count, name):
    """Simple program that greets NAME for a total of COUNT times."""
    for x in range(count):
        click.echo(f"Hello {name}!")

In [ ]:
%%bash
sal --name=Mauro --count=2

Hello Mauro!
Hello Mauro!


In [ ]:
# | export
def is_notebook() -> bool:
    try:
        shell = get_ipython().__class__.__name__
        if shell == "ZMQInteractiveShell" or shell == "CaptureShell":
            return True  # Jupyter notebook or qtconsole
        elif shell == "TerminalInteractiveShell":
            return False  # Terminal running IPython
        else:
            return False  # Other type (?)
    except NameError:
        return False  # Probably standard Python interpreter

In [ ]:
# | export
if __name__ == "__main__" and not is_notebook():
    hello()

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()